Importando e iniciando a cifra de Vigenere, para gerar o alvo do ataque:

In [89]:
from Cifra_Vigenere import *
cifra = Vigenere()
cypher = cifra.cypher

# Começando o ataque

Vamos agora tentar atacar esse método. Primeiro apresentamos um texto maior que possiblita a análise de frequência. A mensagem é a seguinte:

"Throughout history, the art of secret writing has evolved from simple substitution ciphers to the most advanced public key cryptosystems. Early methods relied heavily on letter frequency patterns, making them vulnerable to analytical attacks. Modern cryptography blends mathematics, computer science, and information theory to ensure confidentiality and integrity in a digital world. Researchers constantly probe algorithms to identify weaknesses and strengthen protocols. By experimenting with challenging but known plaintexts, learners develop intuition about how cryptanalysis works in practice. This extended sample contains diverse punctuation marks, spaces, and varying word lengths to simulate a realistic message that still allows for systematic analysis. Users can encrypt this passage using a repeating Vigenere key and then apply their attack strategies to recover the key length and internal structure based on statistical methods."

Perceba que, da forma como foi implementado, o nosso cifrador irá ignorar espaços e pontuação, esse serão preservados na mensagem da forma como estão. No entanto, acentos ortográficos seriam desconsiderados, então Vigenère seria criptografado apenas a parte Vigenre e o è permaneceria como está. No texto acima evitamos essa peculiaridade propositadamente.

Obtemos como resultado o seguinte texto cifrado:

In [90]:
key = "CRYPTOGRAPHY"
message = "Throughout history, the art of secret writing has evolved from simple substitution ciphers to the most advanced public key cryptosystems. Early methods relied heavily on letter frequency patterns, making them vulnerable to analytical attacks. Modern cryptography blends mathematics, computer science, and information theory to ensure confidentiality and integrity in a digital world. Researchers constantly probe algorithms to identify weaknesses and strengthen protocols. By experimenting with challenging but known plaintexts, learners develop intuition about how cryptanalysis works in practice. This extended sample contains diverse punctuation marks, spaces, and varying word lengths to simulate a realistic message that still allows for systematic analysis. Users can encrypt this passage using a repeating Vigenere key and then apply their attack strategies to recover the key length and internal structure based on statistical methods."
language = "en"
setKey(key)
setmsg(message)
cypher()

with open("cypher.bin", "rb") as f:
    encrypted = f.read()
    encrypted = bytearray(encrypted)

encrypted

# Arquivo input.txt cifrado com sucesso e salvo como cypher.bin.


bytearray(b'\x97\xba\xcb\xbf\xc9\xb6\xaf\xc1\xb6\xc4h\xc1\xac\xc5\xcd\xbf\xc6\xc8sr\xb5\xb8\xady\xa4\xc4\xcdp\xc3\xb5g\xc5\xa6\xb3\xba\xbe\xb7r\xd0\xc2\xbd\xc3\xb0\xc0\xa8p\xb0\xba\xb6r\xbe\xc6\xc3\xbb\xbd\xb7\xa5p\xae\xcb\xb2\xbfy\xc3\xbd\xbc\xb7\xbe\xa6p\xbb\xce\xa5\xc5\xcd\xb9\xc8\xc4\xbb\xbb\xb0\xbeh\xbc\xac\xc2\xc1\xb5\xc6\xc2g\xc6\xb0p\xbc\xc1\xa8r\xc6\xbf\xc7\xc3g\xb3\xa5\xc6\xa9\xc7\xa6\xb7\xbdp\xc4\xc4\xa9\xbe\xaa\xb3h\xc4\xa8\xcby\xb3\xc6\xc8\xb7\xc6\xb0\xc3\xc1\xcc\xb7\xb7\xc6\xc3\x82o\x8c\xb3\xb3\xbc\xc1y\xb0\xb7\xcd\xb8\xc3\xb3\xbar\xb3\xb5\xb4\xc2\xa8\xb6y\xb8\xb9\xb0\xbd\xbb\xad\xc9h\xc8\xb1r\xc5\xb5\xc8\xc3\xac\xc4a\xb6\xba\xbe\xb4\xc7\xbe\xbe\xb7\xc8g\xc2\xa2\xc4\xbc\xbe\xb5\xc0\xcc|t\xbc\xa8\xbd\xaa\xbe\xafy\xb7\xba\xbe\xbdt\xc5\xbc\xbe\xaf\xb5\xba\xba\xa5\xbe\xbep\xc8\xbeg\xb3\xaf\xb1\xb4\xd2\xb7\xbb\xbc\xb1\xc0o\xa8\xc6\xb5\xb1\xab\xc4\xb6\x80y\x9d\xc3\xb3\xac\xc4\xafp\xab\xcb\xbc\xc2\xcd\xbf\xbb\xc1\xa8\xc2\xa9\xc9h\xbb\xaf\xb7\xc7\xb4\xc7o\xb4\xb3\xb5\xb8\xad\xc6\

# Descobrindo o tamanho da chave
Agora vamos descobrir o tamanho da chave. Podemos fazer isso analizando combinações de duas, três ou quatro letras, depois verificamos o espaçamento entre essas combinações e computamos os fatores primos desse espaçamento e verificamos aqueles que mais se repetem. Esse é o chamado método de Kasiski.

In [91]:
def factors_up_to(n: int, max_factor: int) -> list[int]:
    if n < 2 or max_factor < 2:
        return []
    
    factors = []
    # Only need to loop up to min(sqrt(n), max_factor)
    upper = min(int(n ** 0.5), max_factor)
    for i in range(2, upper + 1):
        if n % i == 0:
            factors.append(i)
            complement = n // i
            # Add the complementary factor if within limit and not duplicate
            if complement != i and complement <= max_factor:
                factors.append(complement)
    
    # If n itself is <= max_factor, include it
    if n <= max_factor:
        factors.append(n)
    
    return sorted(factors)


In [92]:
from collections import defaultdict

def letters_frequency(encrypted: bytearray, seq_len: int = 3, max_key_size: int = 20):
    positions: defaultdict[bytes, list[int]] = defaultdict(list)
    for i in range(len(encrypted) - seq_len - 1):
        combination = bytes(encrypted[i:i+seq_len])
        positions[combination].append(i)

    factors_frequency: dict[int, int] = {}
    for locs in positions.values():
        if len(locs) > 1:
            for j in range(len(locs)-1):
                spacing = locs[j+1] - locs[j]
                factors = factors_up_to(spacing, max_key_size)
                for factor in factors:
                    if factor not in factors_frequency:
                        factors_frequency[factor] = 1
                    else:
                        factors_frequency[factor] += 1

    return factors_frequency

In [93]:
letters_frequency(encrypted)

{2: 42,
 3: 39,
 4: 39,
 6: 39,
 7: 5,
 9: 8,
 12: 39,
 14: 5,
 18: 8,
 5: 5,
 8: 20,
 10: 5,
 15: 5,
 20: 5,
 16: 14,
 17: 2,
 13: 1,
 19: 3,
 11: 1}

Como podemos ver os números que mais se repetem são o 2, 3, 4, 6 e 12 disparadamente. Se verificásssemos usando combinações de 2 ou 4 letras, obteríamos o mesmo resultado. Isso faz sentido já que o tamanho da nossa chave é 12 e todos os números 2, 3, 4 e 6 são fatores de 12. Na prática poderíamos tentar todas as alternativas ou observar esse fênomeno vendo a discrepância entre o 12 e os demais números (7, 9, 14, etc) e observando que todos os outros números que aparecem com frequência (2, 3, 4, 6) são fatores de 12. Portanto, chegamos no tamanho da nossa chave: 12.

# Descobrindo a chave

Agora sabemos o tamanho da chave (12), podemos então verificar cada letra da chave individualmente. Para cada letra, verificamos a mensagem nos índices 0, 12, 24, etc. Depois 1, 13, 25, etc, e assim por diante. Em cada caso comparamos as frequências dos símbolos com a frequência das letras da língua inglesa e verificamos qual dos deslocamentos mais corresponde. E fazemos assim letra por letra.

In [94]:
def decypher(encrypted_symbol, test_key):
    return (encrypted_symbol - test_key[0]) % 256

def frequency(encrypted: bytearray, key_size: int, key_index: int, test_key: str):
    current_frequency: dict[int, int] = {}
    test_key = bytearray(test_key, "ascii")
    total = 0

    for i in range(key_index, len(encrypted), key_size):
        encrypted_symbol = encrypted[i]
        test_decrypted_symbol = decypher(encrypted_symbol, test_key)
        if test_decrypted_symbol not in current_frequency:
            current_frequency[test_decrypted_symbol] = 1
        else:
            current_frequency[test_decrypted_symbol] += 1

        total += 1

    result: dict[str, float] = {key: value / total for key, value in current_frequency.items()}
    return result

Vamos verificar se uma frequência é "boa" de acordo com a ordem da língua inglesa através de uma comparação estatística entre a frequência das letras da língua inglesa e a frequência das letras advinhadas, aquela que tiver uma pontuação menor de acordo com o teste "chi square" é aquela que tem a frequência mais parecida e será a nossa chave.

In [95]:
def chi_square_statistic(observed: dict[int, float], expected: dict[str, float]) -> float:
    chi2 = 0.0
    for category, E in expected.items():
        O = observed.get(ord(category.lower()), 0.0)
        if E > 0:
            chi2 += (O - E) ** 2 / E
            
    return chi2

In [96]:
english_frequency = {
    "A": 0.08167, "B": 0.01492, "C": 0.02782, "D": 0.04253, "E": 0.12702,
    "F": 0.02228, "G": 0.02015, "H": 0.06094, "I": 0.06966, "J": 0.00153,
    "K": 0.00772, "L": 0.04025, "M": 0.02406, "N": 0.06749, "O": 0.07507,
    "P": 0.01929, "Q": 0.00095, "R": 0.05987, "S": 0.06327, "T": 0.09056,
    "U": 0.02758, "V": 0.00978, "W": 0.02360, "X": 0.00150, "Y": 0.01974,
    "Z": 0.00074,
}
portuguese_frequency = {
    "A": 0.1463, "B": 0.0104, "C": 0.0388, "D": 0.0499, "E": 0.1257,
    "F": 0.0102, "G": 0.0130, "H": 0.0078, "I": 0.0618, "J": 0.0039,
    "K": 0.0002, "L": 0.0278, "M": 0.0474, "N": 0.0505, "O": 0.1073,
    "P": 0.0252, "Q": 0.0120, "R": 0.0653, "S": 0.0781, "T": 0.0434,
    "U": 0.0463, "V": 0.0167, "W": 0.0001, "X": 0.0021, "Y": 0.0001,
    "Z": 0.0047
}

In [97]:
import math, string

def best_key_for_index(encrypted: bytearray, key_size: int, key_index: int, language: str = "en"):
    best_key = None
    best_score = math.inf

    if language == "en":
        freq = english_frequency
    else:
        freq = portuguese_frequency
    
    for test_key in string.ascii_letters:
        score = chi_square_statistic(
            frequency(encrypted, key_size, key_index, test_key), freq
        )
        if score < best_score:
            best_key = test_key
            best_score = score

    return best_key

Vamos testar o nosso método com a primeira letra da chave. Como podemos ver, ele corretamente previu a letra "C" como a primeira.

In [98]:
best_key_for_index(encrypted, 12, 0)

'C'

In [99]:
def attack_cypher(encrypted: bytearray, key_size: int, language: str = "en"):
    result = []
    for i in range(key_size):
        result.append(best_key_for_index(encrypted, key_size, i, language))

    return "".join(result)

E como podemos ver podemos iterar todos os indíces descobrindo a chave.

In [100]:
attack_cypher(encrypted, 12)

'CRYPTOGRAPHY'

Se estivéssemos entre os tamanhos 2, 3, 4, 6 e 12, poderíamos também compará-los usando a análise de frequência.

In [101]:
import string, math
def best_size(encrypted: bytearray, candidates: list[int]) -> int:
    best_candidate = None
    best_score = math.inf

    for key_size in candidates:
        total_chi = 0.0
        for i in range(key_size):
            # find the best shift at this position
            min_chi = math.inf
            for test_key in string.ascii_letters:
                obs = frequency(encrypted, key_size, i, test_key)
                chi = chi_square_statistic(obs, english_frequency)
                if chi < min_chi:
                    min_chi = chi
            total_chi += min_chi

        avg_chi = total_chi / key_size
        if avg_chi < best_score:
            best_score = avg_chi
            best_candidate = key_size

    return best_candidate


In [102]:
best_size(encrypted, [12, 6, 4, 3, 2])

12

Farei uma função agora que irá pegar os melhores candidatos baseado na análise do Kasiski, escolhendo aqueles candidatos depois de uma variação brusca. Por exemplo, perceba que no nosso exemplo anterior, depois do 12 com 38 repetições o segundo mais frequente era o 8 com 15. Essa é uma variação bastante brusca e possivelmente uma boa heurística para selecionar os melhores candidatos.

In [103]:
def select_candidates(encrypted: bytearray, seq_len: int = 3, max_key_size: int = 20):
    text = encrypted
    freqs = letters_frequency(text, seq_len, max_key_size)
    items = sorted(freqs.items(), key=lambda kv: kv[1], reverse=True)
    if len(items) < 2:
        return [items[0][0]] if items else []

    # compute consecutive drops
    drops = [items[i][1] - items[i+1][1] for i in range(len(items)-1)]
    maximum_difference = drops.index(max(drops))

    return [size for size, _ in items[: maximum_difference+1]]

In [104]:
select_candidates(encrypted)

[2, 3, 4, 6, 12]

Finalmente criemos uma função que une todas as técnicas para descobrir uma senha qualquer.

In [105]:
def attack(encrypted: bytearray, language="en"):
    candidates = select_candidates(encrypted)
    size = best_size(encrypted, candidates)
    return attack_cypher(encrypted, size, language=language)

In [106]:
attack(encrypted, language)

'CRYPTOGRAPHY'

Testemos agora com uma outra senha. DIFFERENT

In [107]:
key = "DIFFERENT"
message = "Throughout history, the art of secret writing has evolved from simple substitution ciphers to the most advanced public key cryptosystems. Early methods relied heavily on letter frequency patterns, making them vulnerable to analytical attacks. Modern cryptography blends mathematics, computer science, and information theory to ensure confidentiality and integrity in a digital world. Researchers constantly probe algorithms to identify weaknesses and strengthen protocols. By experimenting with challenging but known plaintexts, learners develop intuition about how cryptanalysis works in practice. This extended sample contains diverse punctuation marks, spaces, and varying word lengths to simulate a realistic message that still allows for systematic analysis. Users can encrypt this passage using a repeating Vigenere key and then apply their attack strategies to recover the key length and internal structure based on statistical methods."
language = "en"
setKey(key)
setmsg(message)
cypher()

with open("cypher.bin", "rb") as f:
    encrypted = f.read()
    encrypted = bytearray(encrypted)

encrypted

# Arquivo input.txt cifrado com sucesso e salvo como cypher.bin.


bytearray(b'\x98\xb1\xb8\xb5\xba\xb9\xad\xbd\xc9\xb8i\xae\xaf\xb8\xc6\xb4\xc0\xcdpi\xba\xae\xaar\xa6\xc0\xc8d\xb8\xacf\xb8\xb7\xa8\xc0\xb9\xb8i\xbd\xb8\xae\xc6\xae\xbc\xbbd\xb1\xa7\xb9e\xb7\xbb\xbd\xc0\xba\xae\xaaf\xab\xc4\xb4\xbbt\xb7\xb2\xb3\xb6\xb1\xb7e\xc1\xc9\xa6\xbc\xba\xaf\xb9\xc7\xb9\xb7\xc3\xb2i\xa9\xaf\xb5\xba\xaa\xc0\xc7d\xbd\xb5f\xb9\xba\xaan\xc1\xb3\xbc\xbaf\xa6\xb6\xbb\xaf\xc2\xa7\xae\xaaf\xb5\xc7\xa7\xba\xbd\xa7i\xb1\xab\xber\xa8\xc0\xcd\xb4\xbd\xb5\xb9\xbe\xc5\xb9\xb3\xc1\xb7wf\x8b\xa6\xc4\xb1\xc7t\xb1\xae\xba\xae\xb4\xb6\xb8n\xc6\xa9\xb5\xaf\xab\xa9r\xad\xb3\xb5\xba\xb2\xb2\xbfe\xc1\xb3n\xc0\xa9\xbd\xba\xab\xb7r\xab\xc0\xb9\xb5\xbe\xab\xb4\xa8\xcbe\xbe\xb5\xb8\xbd\xab\xb8\xb3\xc5qn\xc1\xa5\xb4\xaf\xb4\xacr\xb9\xb6\xb9\xb1i\xbc\xbb\xb1\xc0\xaa\xc0\xb5\xa6\xb5\xabf\xb9\xc1e\xaf\xc2\xa5\xb5\xbf\xba\xae\xb5\xa6\xbat\xa5\xbd\xba\xa7\xa8\xbd\xb8|t\x91\xb8\xaa\xab\xb7\xc0e\xb1\xc6\xbd\xb9\xba\xb5\xac\xc4\xa6\xbe\xbc\xbdi\xa8\xb2\xaa\xc0\xa9\xc1t\xb1\xaa\xba\xae\xaa\xbf\xa6\xc

In [108]:
attack(encrypted)

'DIFFERENT'

Finalmente, testemos com outra mensagem. É importante que ela seja em inglês e que contenha um número de palavras suficientemente grande para que a nossa análise estatística seja bem-sucedida.

In [109]:
message = "Cryptography is the science of securing communication by converting data into unreadable formats, ensuring confidentiality. Modern systems use symmetric ciphers like AES, asymmetric methods such as RSA or elliptic curve cryptography, and hashing functions like SHA. Techniques such as digital signatures verify identity and protect information in transit and at rest from unauthorized access and tampering."
secret = "PASSWORD"
language = "en"
setKey(secret)
setmsg(message)
cypher()

with open("cypher.bin", "rb") as f:
    encrypted = f.read()
    encrypted = bytearray(encrypted)

encrypted

# Arquivo input.txt cifrado com sucesso e salvo como cypher.bin.


bytearray(b'\x93\xb3\xcc\xc3\xcb\xbe\xb9\xb6\xb1\xb1\xbb\xccw\xb8\xc5d\xc4\xa9\xb8s\xca\xb2\xbb\xa9\xbe\xa4\xb8s\xc6\xb5r\xb7\xb5\xa4\xc8\xc5\xc0\xbd\xb9d\xb3\xb0\xc0\xc0\xcc\xbd\xbb\xa7\xb1\xb5\xbc\xc2\xc5o\xb4\xbdp\xa4\xc2\xc1\xcd\xb4\xc4\xb8\xb9\xaf\xbas\xbb\xb0\xc6\xa5p\xaa\xc1\xc7\xc6o\xc7\xb2\xc2\xa6\xb4\xb7\xb8\xb1\xbe\xa9p\xa7\xc2\xc5\xc4\xb0\xc6\xb7|a\xb8\xc1\xca\xc4\xc4\xad\xbe\xa8s\xb6\xc6\xbd\xb8\xad\xb4\xa6\xc1\xc7\xc0\xb0\xbe\xad\xc4\xba\x81s\xa4\xbe\xb6\xa9\xc2\xafs\xc6\xd0\xc2\xc6\xa9\xbd\xb4s\xc8\xca\xb4r\xb7\xc9\xae\xc0\xb8\xcb\xc1\xbb\xa7p\xa4\xbc\xc3\xbf\xb4\xc4\xb7p\xad\xbc\xbe\xbco\x93\x89\xa3ms\xb4\xca\xc8\xbf\xb1\xb5\xb5\xc5\xbc\xbao\xbf\xa9\xc4\xa9\xc2\xb7\xcao\xc5\xb9\xb3\xa9s\xb4\xcao\xa4\x97\x91a\xc2\xc5w\xb4\xbe\xb0\xb9\xb1\xc7\xbc\xbao\xb5\xb9\xc2\xb7\xb8s\xba\xc1\xcb\xb4\xc4\xb0\xba\xc5\xb8\xbf\xba\xbd|a\xb4\xc1\xbbo\xba\xa5\xc3\xa9\xbc\xc1\xbeo\xb8\xb9\xbe\xa4\xc7\xbc\xc6\xbd\xc5d\xbc\xaa\xbe\xb8w\xa2\x9a\x85~a\xa7\xb8\xba\xb7\xc0\xad\xc1\xb6\xb8\xc6w\xc

In [110]:
select_candidates(encrypted)

[2, 4, 8]

In [111]:
attack(encrypted, language)

'PASSWORD'